<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 5. Interactive Pipelines</h2></div>

In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
import panel as pn

pn.extension('tabulator', template='material')

pn.state.template.sidebar_width = 250
pn.config.sizing_mode = 'stretch_width'

import hvplot.pandas

## Panel widgets

In this notebook, we will want to drive our visualizations with interactive widgets. For this, we will need a widget library where we will be using [panel](https://panel.holoviz.org/).

### Float slider

For instance, let us create a float slider to specify an earthquake magnitude between zero and nine:

In [ ]:
mag_slider = pn.widgets.FloatSlider(name='Magnitude', start=0, end=9, value=6)
mag_slider

We can get the value of this slider from the `.value` parameter:

In [ ]:
mag_slider.value

Try moving the slider around and rerunning the cell above to access the current slider value.

## Date range slider

Panel has many different widgets and you can see a reference gallery of them [here](https://panel.holoviz.org/reference/index.html#widgets). Let's now make a widget to specify a date range:

In [ ]:
date_range = pn.widgets.DateRangeSlider(name='Date', 
                                        start=pd.Timestamp('2000-01-31'), 
                                        end=pd.Timestamp('2018-12-01'))
date_range

Now we can see the value of this slider:

In [ ]:
date_range.value

As it is a range, this time you get a tuple. You can get the components of the start and end with `value_start` and `value_end` respectively:

In [ ]:
f'Start is at {date_range.value_start} and the end is at {date_range.value_end}'

Again try specifying different ranges with the widgets and rerunning the cell above.

## The `.interactive` interface

To use the .`interactive` interface from `hvplot`, first we load our earthquake data as before:

In [ ]:
df = pd.read_parquet('../data/earthquakes.parq')
df = df.set_index('time').tz_convert(None).reset_index()
df.head(n=3)

After importing `hvplot.pandas`, we now have an `.interactive` method on our `DataFrame`, allowing us to create an *interactive* `Dataframe`. Here we specify a `sizing_mode` to control how our visualizations will look:

In [ ]:
dfi = df.interactive(sizing_mode='stretch_width')
print(dfi) # TODO: hangs without print

This behaves just like a regular `DataFrame` except now you can pass panel *widgets* as arguments to the pandas methods you are familiar with. Let us make the same magnitude and date range widgets where the only difference is that the start and end dates can now be constrained by the data:

In [ ]:
mag = pn.widgets.FloatSlider(name='Magnitude', start=0, end=9, value=6)  # MINIMUM MAGNITUDE
date = pn.widgets.DateRangeSlider(name='Date', start=df.time.iloc[0], end=df.time.iloc[-1])

Now we can pass these widgets to a simple filtering function where we filter a pandas `DataFrame` in the standard way. The only difference is that now we can refer to the parameter values of our widgets instead of only specifying literal values. You can refer to the `value`, `value_start` and `value_end` parameters of the widgets with `param.value`, `param.value_start` and `param.value_end` respectively:

In [ ]:
filtered = dfi[
    (dfi['mag']   > mag) &
    (dfi['time'] >= date.param.value_start) &
    (dfi['time'] <= date.param.value_end)
]

filtered.head()

Now we have a pandas `DataFrame` but on top we also have the two widgets we declared. When the widgets are interacted with, the `DataFrame` updates.

*Note that to see the table update, you want to move the start date of the range slider. Otherwise, you won't see much change as the earthquakes are shown ordered by date.*

For the magnitude, we can just specify `mag` as the `value` parameter is used by default when a widget is supplied. You can of course use, `mag.param.value` instead if you wish. Try it!

## Plotting with `.interactive`

All pandas methods can be made interactive this way, including the built in `.plot` method using matplotlib:

In [ ]:
filtered.plot(y='depth', kind='hist', bins=np.linspace(0, 50, 51))

# TODO:

In [ ]:
mag_hist = filtered.hvplot(
    y='mag', kind='hist', responsive=True, min_height=200
)

depth_hist = filtered.hvplot(
    y='depth', kind='hist', responsive=True, min_height=200
)

mag_hist + depth_hist